In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
# 날짜를 다루는 라이브러리
from datetime import date
import os
# from dotenv import load_dotenv
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()


In [6]:
# 상장공시시스템
page = 1
a = True
dic = {'상장종류' : [], "회사명" : [], "업종" : [], "주요제품" : [], "상장일" : [], "결산월" : [], "대표자명" : [], "홈페이지" : [], "지역" : []}
while a :
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = {'method' : 'searchCorpList', 'pageIndex' : page, 'currentPageSize' : 100, 'orderMode' : 3, 'orderStat':'D', 'searchType':13}
    r = requests.get(url, params = payload)
    

    data = r.text
    soup = BeautifulSoup(data, 'lxml')
    
    total_items = int(soup.select_one(".info.type-00 > em").text.replace(",",""))
    total_pages = total_items // 100 + 1
    print(f"{page}/{total_pages} 수집중", end = "\r")

    # 전체 페이지
    # 상장종류
    types = []
    for i in soup.select('td.first > img:nth-of-type(1)') :
        types.append(i.get('alt'))

    # 회사명
    com_name = []
    for i in soup.select('td.first > a') :
        com_name.append(i.string)

    # 업종 & 주요제품
    sums = []
    sector = []
    product = []
    for i in soup.select('.textOverflow') :
        sums.append(i.string)

    for sec in sums[0::2] :
        sector.append(sec)
    for pro in sums[1::2] :
        product.append(pro)
    # sector와 product를 딕셔너리에 setdefault로 넣으면 됨

    # txc에 엮인 것들 모조리 가져오기
    entire = []
    ceo_names = []
    for i in soup.find_all('td', 'txc') :
        if i.find('a') is not None :
            entire.append(i.find('a')['href'])
        elif i.string == '\n' :
            entire.append("None")
        elif i is not None :
            entire.append(i.string)

    # 대표명은 string이 아닌 title 값을 가져와야 함
    for i in soup.find_all('td', class_='txc') :
        ceo_names.append(i.get('title'))
        
    # title 값으로 가져온 대표명을 다시 entire의 대표명으로 교체   
    for i in range(0, len(entire)) :
        if i % 5 == 2 :
            entire[i] = ceo_names[i]

    # 상장일, 결산월, 대표명, 홈페이지, 지역
    date = []
    month = []
    ceo_name = []
    homepage = []
    region = []
    for i in range(0,5) :
        if i == 0 :
            for da in entire[i::5]:
                date.append(da)
        if i == 1 :
            for mo in entire[i::5]:
                month.append(mo)
        if i == 2 :
            for ceo in entire[i::5]:
                ceo_name.append(ceo)
        if i == 3 :
            for home in entire[i::5]:
                homepage.append(home)
        if i == 4 :
            for re in entire[i::5]:
                region.append(re)

    # 딕셔너리에 페이지에서 가져온 값들 value로 넣기
    for key in dic :
        if key == "상장종류" :
            dic[key].extend(types)
        if key == "회사명" :
            dic[key].extend(com_name)
        if key == "업종" :
            dic[key].extend(sector)
        if key == "주요제품" :
            dic[key].extend(product)
        if key == "상장일" :
            dic[key].extend(date)
        if key == "결산월" :
            dic[key].extend(month)
        if key == "대표자명" :
            dic[key].extend(ceo_name)
        if key == "홈페이지" :
            dic[key].extend(homepage)
        if key == "지역" :
            dic[key].extend(region)
    # 전체 페이지까지 진행되도록 하는 코드
    if page < total_pages :
        print('page :', page, end = "\r")
        page += 1
        time.sleep(0.5)
    else :
        a = False

df = pd.DataFrame(dic)
df

,상장종류,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,아이비젼웍스,특수 목적용 기계 제조업,이차전지 검사시스템 및 검사시스템 설치셋업용역,2024-09-03,12월,길기재,http://www.ivw.co.kr/kor/main/,대전광역시
1,코스닥,아이스크림미디어,소프트웨어 개발 및 공급업,"교육출판, 커머스, 연수",2024-08-30,12월,허주환,http://www.i-screammedia.com,경기도
2,코스닥,이엔셀,기초 의약물질 제조업,"첨단바이오의약품 위탁개발생산(CDMO) 서비스 / 차세대 세포,유전자 치료제",2024-08-23,12월,장종욱,http://www.encellinc.com/,서울특별시
3,코스닥,M83,"영화, 비디오물, 방송프로그램 제작 및 배급업",VFX,2024-08-22,12월,"정성진, 김호성",http://m83.co.kr,서울특별시
4,코스닥,대신밸런스제18호스팩,금융 지원 서비스업,금융 지원 서비스업,2024-08-22,12월,문병식,None,서울특별시
...,...,...,...,...,...,...,...,...,...
2710,유가증권,유한양행,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이사 조욱제,http://www.yuhan.co.kr,서울특별시
2711,유가증권,CJ대한통운,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,"신영수, 민영학 (각자 대표)",http://www.cjlogistics.com,서울특별시
2712,유가증권,경방,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준, 김담",http://www.kyungbang.co.kr,서울특별시
2713,유가증권,유수홀딩스,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시
